In [112]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from keras import layers
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [167]:
img_with_tumor_dir = '/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes'
img_without_tumor_dir = '/kaggle/input/brain-mri-images-for-brain-tumor-detection/no'

In [170]:
print('There are {} images with tumor'.format(len(img_with_tumor_dir)))

There are 60 images with tumor


In [171]:
print('There are {} images without tumor'.format(len(img_without_tumor_dir)))

There are 59 images without tumor


In [207]:
img_height, img_width = 224, 224
batch_size = 16
epochs = 30
learning_rate = 0.0001
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001)

In [192]:
# Image data generator with rescaling
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training data generator
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

Found 406 images belonging to 3 classes.
Found 100 images belonging to 3 classes.


In [193]:
# load base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

In [210]:
# Freeze the layers which you don't want to train
for layer in base_model.layers[-10:]:
    layer.trainable = False


# Create a new model
model = Sequential()

# Add the VGG16 base model
model.add(base_model)

# Add new custom layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))





In [201]:
# Option 1: Use RMSprop
optimizer_1 = RMSprop(learning_rate=learning_rate)

# Option 2: Use SGD with momentum
optimizer_2 = SGD(learning_rate=learning_rate, momentum=0.9)

# Option 3: Use Adam 
optimizer_3 = Adam(learning_rate=learning_rate)

In [213]:
# Compile the model
model.compile(optimizer=optimizer_3, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_40 (Flatten)            │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,252,933 (203.14 MB)

 Trainable params: 12,846,081 (49.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 25,692,164 (98.01 MB)

In [214]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#class_weight = {0: 1., 1: 1.5}  


history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    #class_weight=class_weight,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/30


ValueError: Unknown variable: <KerasVariable shape=(25088, 512), dtype=float32, path=sequential_26/dense_71/kernel>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should recreate a new optimizer instance.

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

In [ ]:
# Plot training & validation accuracy and loss values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
# Example function to predict tumor presence
def predict_tumor(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        print("Tumor detected.")
    else:
        print("No tumor detected.")